# CS202: Compiler Construction

## In-class Exercises, Week of 03/20/2023

----

# Part 1: While Loops

## Question 1

Compile the following program to $\mathcal{C}_{if}$:

```
i = 10
sum = 0
while i > 0:
    i = i - 1
    sum = sum + i
print(sum)
```

Output of RCO:
```
i = 10
sum = 0
tmp_1 = i > 0
while tmp_1:
    (compilet statements here)
```
(this means we'll need to do something special in rco)

CIF version:
```
start:
    i = 10
    sum = 0
    tmp_1 = i > 0
    goto while_test
while_test:
    if tmp_1 then goto while_body else goto cont
while_body:
    i = i - 1
    sum = sum + i
    goto while_test
cont:
    print(sum)
```

## Question 2

Compile the program above into pseudo-x86 assembly.

```
start:
    movq $10, #i
    movq $0, #sum
    jmp while_test
while_test:
    cmpq $0, #i
    jgt while_body
    jmp cont
while_body:
    subq $1, #i
    addq #i, #sum
    jmp while_test
cont:
    movq #sum, %rdi
    callq print_int
```

## Question 3

Describe the major changes to the compiler, up to *select instructions*.

No new passes.

- Typechecker
    - Add a case for while loops
    - Condition must be a bool
    - Statements must be well typed
- RCO
    - Add case for while loops
    - Easy part: run rco_stmts on the body statements of the loop
    - Hard part: condition
        - Problem: tmp vars created by rco_exp end up outside the loop
        - Solution:
            - Construct new bindings just for tmp vars associated with the condition
            - Package up resulting Assign statements into a Begin expression
                - cond_bindings = {}
                - new_cond_exp = rco_exp(cond, cond_bindings)
                - create a begin node with list of assignment statements for everything in `cond_bindings`, and the expression `new_cond_exp`
- Explicate-control
    - new case: While(Begin(cond_stmts, cond_exp), body_stmts)
    - creat a loop shaped control flow graph
        - cont_label = use `create_block` to add `cont` to the CFG
        - test_label = gensym('loop_label')
        - body_label = use `create_block` to add the result of compiling `body_stmts` to the CFG (liek the `then` case for if). Continuation should be `[cif.Goto(test_label)]`
        - compile the test:
            - let the cont be `[cif.If(explicate_exp(cond_exp, cif.Goto(body_label), cif.Goto(cond_label)))]`
            - compile cond_stmts with this continuation
            - `basic_blocks[test_label] = ` result from above
        - return the new continuatin: `[cif.Goto(test_label)]`
- Select-instructions: no changes

# Part 2: Dataflow Analysis

## Question 4

Perform liveness analysis on the pseudo-x86 program from Question 2.

Use approach from A3: When we find a jmp, go do the liveness analysis on the target to get its live before set

```
start:
    movq $10, #i {}
    movq $0, #sum {}
    jmp while_test {}
while_test: {}
    cmpq $0, #i {}
    jgt while_body {sum}
    jmp cont {}
while_body: {}
    subq $1, #i {}
    addq #i, #sum {}
    jmp while_test {}
cont:              {sum}
    movq #sum, %rdi {}
    callq print_int {}
```

Problem: to compute the live before of while_body, we need the live before of while_test; to compute live before of while_test, we need hte live before of while_body. infinite loop

## Question 5

Describe the idea of dataflow analysis on cyclic control-flow graphs.

1. compute the live-after sets for each block without worrying about jmps(assume all live-before sets are empty). This is an **underapproximation** of the true live-after sets.
2. Update the live-before sets based on the results from #1
3. Run #1 until the live-after sets don't change at all. This is called a **fixpoint**.

## Question 6

Use the dataflow-based approach to perform liveness analysis on the pseudo-x86 program above.

```
start:           {}
    movq $10, #i {i}
    movq $0, #sum {i, sum}
    jmp while_test {}
while_test:    {i, sum}
    cmpq $0, #i {i, sum}
    jgt while_body {sum}
    jmp cont {}
while_body:   {i, sum}
    subq $1, #i {i, sum}
    addq #i, #sum {i, sum}
    jmp while_test {}
cont:              {sum}
    movq #sum, %rdi {}
    callq print_int {}
```

Fter 4 iterations nothing changes, so done

Changeto the liveness analysis in the compiler:
- add a ul_fixpoint function
- while loop:
    - make a copy of the current live after sets
    - run ul_block on each block of teh program
    - exit while loop if the live after sets are the same as the copy
    
- initialize the live before sets to be empty for *all* blocks
- remove call to ul_block in the jmp case

## Question 7

How do we know the dataflow analysis will stop (i.e. not loop forever)?

Two big questions:
- What is the live-after sets keep changing? Then ul_fixpoint runs forever
    - There are finitely many variables in the program, and finitely many blocks
    - In the worst case, every variable ends up in every live-after set
    - At that point, there is nothing we can add to any set, so the state can't possibly change if we run the analysis again
- We initialize live-befores *wrong*, so how do we know the final answer will be correct?
    - Imagine if we knew the correct live-before set of each label
    - Then we could run liveness analysis on the blocks in any order, and get the right answer
    - Imagine that some live-before set is missing a variable that should be there
    - Because all other live-before sets are correct, the next iteration of liveness analysis will fill in that missing variable of tha live-before set

## Question 8

What changes are required for the rest of the compiler?